In [ ]:
# Import libraries
import os
import pandas as pd

# Import nanoHUB-specific libraries
import nanohubremote as nr
from simtool import findInstalledSimToolNotebooks,searchForSimTool
from simtool import getSimToolInputs,getSimToolOutputs,Run



In [ ]:
# Create a nanoHUB web services session
auth_data = {
    'grant_type' : 'tool',
}
with open(os.environ["SESSIONDIR"]+"/resources") as file:
    lines = [line.split(" ", 1) for line in file.readlines()]
    properties = {line[0].strip(): line[1].strip() for line in lines if len(line)==2}
    auth_data["sessiontoken"] = properties["session_token"]
    auth_data["sessionnum"] = properties["sessionid"]
    
session = nr.Sim2l(auth_data)

In [ ]:
# Query the vaspingestor tool for all runs submitted by Kat Nykiel
tool = 'vaspingestor'

installedSimToolNotebooks = findInstalledSimToolNotebooks(tool,returnString=True)
print(installedSimToolNotebooks)
cellrelaxdft = searchForSimTool(tool)

req_json = session.requestPost('dbexplorer/dbexplorer/search?simtool=true', data={ 'filters': '[{"field": "input.author", "operation": "=", "value": "Kat '
             'Nykiel"}]',
  'limit': 100,
  'results': '["output.structure", "output.composition", '
             '"output.lattice_parameters", "output.lattice_angles", '
             '"output.energy", "output.forces", "output.max_force", '
             '"output.rms_force", "output.KPOINTS", "output.ENCUT", '
             '"output.XC_functional"]',
  'revision': 0,
  'tool': 'vaspingestor'}, timeout=20) # QUERY
req_json = req_json.json()
data = pd.DataFrame(req_json['results']).dropna().reset_index(drop=True) # Deleting columns with NaNs (you can comment this out)
data.head()

In [ ]:
data.shape